In [1]:
# !pip install parse
# !pip install matplotlib --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

colab_datadir = '/content/drive/MyDrive/sharedCodeBase/FamilyAnalysis/'
local_datadir = './'
datadir = local_datadir
import sys
sys.path.append(datadir)

In [2]:
import pandas as pd
import numpy as np

from parse import *
import os
import sklearn
from sklearn.datasets import make_blobs
from matplotlib import pyplot as plt
#%matplotlib auto
import matplotlib.image as mpimg 
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.collections import PolyCollection
from collections import defaultdict
import datetime
from datetime import timedelta
import argparse
from myHelper import *
from graphHelper import *

# 选择家庭



In [3]:
class Myclass():
    MinCountOfPoints=6 #这里我没改，因为分长时间和短时间计算时，我手动一起调了（详见最后一个part）
    MinCountOfPointsForAll=-1
    SizeOfGrid='0.5' # 不需要调 
    figure=datadir+'floorplan01.png' # 默认
    path=datadir+'results_both/' # 默认
    res=datadir+'figures_both/' # 默认
    bySecond=False
    IntervalOfMinutesOrSeconds=1 # 1分钟 1个点  
    setConfig=True #False

arguments=Myclass()

##########################################
# Choose Family
##########################################
family_config = chooseFamily()

max_x = family_config.size[1]
min_x = family_config.size[0]
max_y = family_config.size[3]
min_y = family_config.size[2]

#需要改（mytask和advanced都需要改）
if family_config.name == "ParentsOfSun":
  arguments.figure = datadir+'floorplan01.png'
  arguments.path = datadir+'family1/results_tag0/'
  arguments.res=datadir+'family1/figures_tag0/'
elif family_config.name == "UncleOfSun":
  arguments.figure = datadir+'floorplan02.png'
  arguments.path = datadir+'family2/results_tag0/'
  arguments.res=datadir+'family2/figures_tag0/'
elif family_config.name == "AuntOfSun":
  arguments.figure = datadir+'floorplan03.png'
  arguments.path = datadir+'family3/results_tag2/'
  arguments.res=datadir+'family3/figures_tag2/'
else:
  print("Family doesn't exist !\n")
  exit(0)

unit = float(arguments.SizeOfGrid)  #网格单元长宽m
column_num = int((max_x-min_x)//unit)
row_num = int((max_y-min_y)//unit)
map_img = mpimg.imread(arguments.figure)

# time epsilon to select the data point
epsilon = 5 #timedelta(minutes=10)
min_count = arguments.MinCountOfPoints

# define time
year = 2022
input_path = arguments.path
res_path = arguments.res
C = 0

==>Familiy choices :<==
1 :ParentsOfSun
2 :UncleOfSun
3 :AuntOfSun
4 :Zeng
Family of ParentsOfSun is chosen!

Trace familes:
	ParentsOfSun
	size of house : [-2.665, 13.505, -2.2, 10.44]
		Scenario=Shift
			zone 1: OutDoor
				{ x : [-1000.0,-0.0],y : [-1000.0,1000.0] => Delete
			zone 2: OutCanteen
				{ x : [-0.415,1.17],y : [6.773,1000.0] => MoveToDown
			zone kitchen: InKitchen
				{ x : [1.17,3.6],y : [6.785,10.44] => MoveToLeftLinearly
			zone 3: OutKitchen
				{ x : [1.17,3.6],y : [10.44,1000.0] => MoveToDown
			zone 4: OutKitchen
				{ x : [3.6,1000.0],y : [10.44,1000.0] => MoveToLeftDown
			zone 5: OutCanteen
				{ x : [3.6,1000.0],y : [5.37,6.785] => MoveToLeft
			zone 6: OutBalcony
				{ x : [-0.415,6.8],y : [-1000.0,-2.2] => MoveToUp
			zone 7: OutBedRoom
				{ x : [4.115,1000.0],y : [1.085,5.37] => MoveToLeft


# Setup

## Read choosed data

Data Format (index,time,x,y,label,zone)

In [4]:
#这里需要改filename = "UncleOfSun_choice_data_all.xlsx"，每换一户需要改一次；同一户里换tag不需要改
filename = "ParentsOfSun_choice_data_all.xlsx" # 可以修改的输入数据路径
df = pd.read_excel(open(datadir+filename,'rb'))
# Drop first column of dataframe
df = df.iloc[: , 1:]
# Drop wrong labels (再次检查-1 label)
df = df.drop(df[df.label.isin([-1])].index)
df

,time,x,y,label,zone
0,2022-02-07 08:29:00,3.054000,6.464179,556,0
1,2022-02-07 08:31:00,2.466070,6.910877,587,2
2,2022-02-07 08:32:00,2.624581,6.069367,523,0
3,2022-02-07 08:33:00,1.871939,4.309019,393,0
4,2022-02-07 08:34:00,3.600000,8.062000,653,2
...,...,...,...,...,...
4978,2022-02-09 22:49:00,2.038864,1.038186,202,1
4979,2022-02-09 22:50:00,1.977453,1.029235,202,1
4980,2022-02-09 22:51:00,2.096200,1.075524,202,1
4981,2022-02-09 22:52:00,1.973830,1.096821,202,1


## Configuration

设置划分常住区域和非常住区域的百分比

In [5]:
#需要划分常驻和非常驻的百分比，意思是网格密度大于各区域最高网格密度 X% 的则是常驻
percentage = 0.1

In [6]:
df["label"].value_counts()

461    741
201    622
202    444
169    234
170    168
      ... 
231     16
525     15
263     14
425     14
589     13
Name: label, Length: 80, dtype: int64

# Normalization ： 对每个区域的每个网格的点数进行归一化处理

## 对数据分组

In [7]:
df_groupby = df.groupby(by=["zone","label"])
df_count = df_groupby.count().rename(columns = {"x" : "number"})
df_count

time  number   y
zone label                  
0    330       7       7   7
     360       8       8   8
     361      25      25  25
     362      53      53  53
     393      18      18  18
...          ...     ...  ..
3    9        24      24  24
     10       52      52  52
     41       28      28  28
     42       84      84  84
     43       37      37  37

[87 rows x 3 columns]

## 对于出现在不同区的相同label，我们进行整理，选择数量更多的zone保存该label

In [8]:
df_count_index = df_count.reset_index() # 重新整理表格
# 对处于不同zone的相同label进行处理
new_df_count = (df_count_index.sort_values("number",ascending=False)
                  #.drop_duplicates(subset=["label"])
                  .groupby(['label']).agg({"number":"sum","zone":"first"})
                  .reset_index())
# 按照降序打印
new_df_count.sort_values("number",ascending=False)

,label,number,zone
45,461,741,0
11,201,622,1
12,202,444,1
7,169,234,1
8,170,168,1
...,...,...,...
76,746,16,2
54,525,15,0
36,425,14,0
18,263,14,1


## 对数据进行归一化处理

In [9]:

# df_normalized = new_df_count.groupby("zone").apply(lambda x: x.number / x.number.max())

zone_list = new_df_count.zone.unique()
grouped = new_df_count.groupby('zone')
split_dfs = []
for zone in zone_list:
  current_group  = grouped.get_group(zone)
  current_group["number"] = current_group.number/current_group.number.max()
  split_dfs.append(current_group)
df_normalized = pd.concat(split_dfs)

/tmp/ipykernel_69728/27825829.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_group["number"] = current_group.number/current_group.number.max()


In [10]:
print(df_normalized)

    label    number  zone
0       9  0.285714     3
1      10  0.619048     3
2      41  0.333333     3
3      42  1.000000     3
4      43  0.440476     3
..    ...       ...   ...
75    717  0.309278     2
76    746  0.164948     2
77    747  0.288660     2
78    748  0.195876     2
79    780  0.195876     2

[80 rows x 3 columns]


## 添加其他信息进入表格

In [11]:
df_normalized["category"] = np.where(df_normalized['number'] > percentage, "常驻", "非常驻")
print(df_normalized)

    label    number  zone category
0       9  0.285714     3       常驻
1      10  0.619048     3       常驻
2      41  0.333333     3       常驻
3      42  1.000000     3       常驻
4      43  0.440476     3       常驻
..    ...       ...   ...      ...
75    717  0.309278     2       常驻
76    746  0.164948     2       常驻
77    747  0.288660     2       常驻
78    748  0.195876     2       常驻
79    780  0.195876     2       常驻

[80 rows x 4 columns]


## 统计常驻和非常驻面积

In [12]:
df_size = df_normalized.groupby(['zone','category']).count().reset_index().drop("label",axis=1)
df_size["size"] = df_size["number"]*unit**2
df_size

,zone,category,number,size
0,0,常驻,2,0.50
1,0,非常驻,26,6.50
2,1,常驻,8,2.00
3,1,非常驻,16,4.00
4,2,常驻,23,5.75
5,3,常驻,5,1.25


# 绘制常驻和非常驻区域

In [13]:
# Define colors 定义颜色
number_zones = len(df_normalized["zone"].unique())
interval = 1/number_zones
dict_color = {}
for i in range(number_zones):
  dict_color["常驻"+str(i)] = (i+1) * interval
  dict_color["非常驻"+str(i)] = (i+0.5) * interval

dict_color

{'常驻0': 0.25,
 '非常驻0': 0.125,
 '常驻1': 0.5,
 '非常驻1': 0.375,
 '常驻2': 0.75,
 '非常驻2': 0.625,
 '常驻3': 1.0,
 '非常驻3': 0.875}

In [14]:

# re-organize the grid
zone_grids = np.zeros([row_num,column_num])
grids = np.zeros([row_num,column_num])
# for i in df_normalized.label:
for index, row in df_normalized.iterrows():
    # parameters 
    i = int(row["label"])
    n = float(row["number"])
    z = int(row["zone"])
    # calculation
    r = (i-1)//column_num 
    c = i - column_num * r - 1 
    # row = df_normalized.loc[df_normalized.index[df_normalized["label"]==i]]
    if n>percentage:
      grids[r,c] = dict_color["常驻"+str(z)]   #0.8是颜色信息，深色
    else:
      grids[r,c] = dict_color["非常驻"+str(z)]  #0.2是颜色信息，浅色

    zone_grids[r,c] =  z

fig, ax = plt.subplots(figsize=(max_x-min_x,max_y-min_y))
ax = sns.heatmap(grids, cmap='coolwarm',  
                  linewidths=0,alpha=0.8, 
                  xticklabels =False,square = True,
                  annot=zone_grids,yticklabels =False,
                  mask=(grids==0.),center=0.5)   #cmap是颜色库；center=表示0.几是颜色区域中心点 #YlGnBu #rocket #coolwarm #cubehelix
ax.invert_yaxis()
#plt.show()
plt.imshow(map_img,zorder = 0, extent= [-0.1,(max_x-min_x)/unit-0.2,-0.1,(max_y-min_y)/unit-0.3])# [x_left/unit,x_right/unit,y_down/unit,y_up/unit])
parsed = parse("{}.{}",filename)
prefix = parsed[0]
fig.savefig(res_path+"analysis1_"+prefix+".png",dpi=300)
plt.close(fig)

# 绘制时间轴(需要配合TimeLineGantt程序使用)

## Re-formalization

In [15]:
# Original data
df

,time,x,y,label,zone
0,2022-02-07 08:29:00,3.054000,6.464179,556,0
1,2022-02-07 08:31:00,2.466070,6.910877,587,2
2,2022-02-07 08:32:00,2.624581,6.069367,523,0
3,2022-02-07 08:33:00,1.871939,4.309019,393,0
4,2022-02-07 08:34:00,3.600000,8.062000,653,2
...,...,...,...,...,...
4978,2022-02-09 22:49:00,2.038864,1.038186,202,1
4979,2022-02-09 22:50:00,1.977453,1.029235,202,1
4980,2022-02-09 22:51:00,2.096200,1.075524,202,1
4981,2022-02-09 22:52:00,1.973830,1.096821,202,1


In [16]:
# 归一化数据
df_normalized

,label,number,zone,category
0,9,0.285714,3,常驻
1,10,0.619048,3,常驻
2,41,0.333333,3,常驻
3,42,1.000000,3,常驻
4,43,0.440476,3,常驻
...,...,...,...,...
75,717,0.309278,2,常驻
76,746,0.164948,2,常驻
77,747,0.288660,2,常驻
78,748,0.195876,2,常驻


In [17]:
df_join = df.join(df_normalized.set_index('label'), on='label', lsuffix='_before', rsuffix='_after')
# 合成新的zone tag
df_join["zone_tag"] = df_join.apply(lambda row: str(row.zone_after) + row.category, axis=1)
df_join


,time,x,y,label,zone_before,number,zone_after,category,zone_tag
0,2022-02-07 08:29:00,3.054000,6.464179,556,0,0.412371,2,常驻,2常驻
1,2022-02-07 08:31:00,2.466070,6.910877,587,2,0.463918,2,常驻,2常驻
2,2022-02-07 08:32:00,2.624581,6.069367,523,0,0.032389,0,非常驻,0非常驻
3,2022-02-07 08:33:00,1.871939,4.309019,393,0,0.024291,0,非常驻,0非常驻
4,2022-02-07 08:34:00,3.600000,8.062000,653,2,0.525773,2,常驻,2常驻
...,...,...,...,...,...,...,...,...,...
4978,2022-02-09 22:49:00,2.038864,1.038186,202,1,0.713826,1,常驻,1常驻
4979,2022-02-09 22:50:00,1.977453,1.029235,202,1,0.713826,1,常驻,1常驻
4980,2022-02-09 22:51:00,2.096200,1.075524,202,1,0.713826,1,常驻,1常驻
4981,2022-02-09 22:52:00,1.973830,1.096821,202,1,0.713826,1,常驻,1常驻


In [18]:
df_join[3554:3566]

,time,x,y,label,zone_before,number,zone_after,category,zone_tag
3554,2022-02-10 19:50:00,1.040000,0.541000,168,1,0.188103,1,常驻,1常驻
3555,2022-02-10 19:51:00,1.172117,0.660933,168,1,0.188103,1,常驻,1常驻
3556,2022-02-10 19:52:00,1.436367,0.670017,169,1,0.376206,1,常驻,1常驻
3557,2022-02-10 19:53:00,1.549637,0.586252,169,1,0.376206,1,常驻,1常驻
3558,2022-02-10 20:03:00,1.000905,0.733664,168,1,0.188103,1,常驻,1常驻
3559,2022-02-10 20:04:00,1.124250,0.759350,168,1,0.188103,1,常驻,1常驻
3560,2022-02-10 20:05:00,1.864154,2.319595,265,1,0.049839,1,非常驻,1非常驻
3561,2022-02-10 20:06:00,3.222297,5.704590,492,0,0.072874,0,非常驻,0非常驻
3562,2022-02-10 20:07:00,1.169617,1.507317,232,1,0.104502,1,常驻,1常驻
3563,2022-02-10 20:08:00,0.921770,0.795644,168,1,0.188103,1,常驻,1常驻


## 新的时间轴处理函数

In [19]:
"""
Time line
"""
def createTimeLineAdvanced(key,details,res_path):
    timeline = []
    for zone in details:
        for t in details[zone]:
            timeline.append((t[0], t[1], zone))

    storeTimeLine(timeline,res_path+key+"_time_advanced.xlsx")

In [21]:
# 需要改 createTimeLine("All",details,res_path)需要更改"ALL"，区分长、中、短、全部
'''
获得{value=[time],key=所属分区}的字典
'''
dict_zones_all = df_join.groupby('zone_tag')['time'].apply(list).to_dict()
#print(dict_zones_all)
"""
get data_allInOne
"""

"""
get the data of corresponding zone
intervals: key=zone名称 value=最大连续停留时长
details: key=zone名称 value=所有停留时段 times_in_zone=[[start1,end1],[start2,end2],.....]
"""
intervals = {}
details = {}
for zone in dict_zones_all:
  intervals[zone],details[zone] = sliceByTime(dict_zones_all[zone],epsilon)

#print(details)
createTimeLineAdvanced("All",details,res_path)


In [22]:
#print(details['2.0常驻'])


In [23]:
#print(dict_zones_all['2.0常驻'])



# new_grids = createHeatmapByTime(dict_zones_all,"heatmap_allInOne",map_img,res_path,row_num,column_num,unit,max_x,min_x,max_y,min_y,groups)
# createHeatmapByTime2(new_grids,"heatmap_allInOne",map_img,res_path,unit,max_x,min_x,max_y,min_y)
